In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Lambda, Dense, Flatten, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.models import load_model
from keras.callbacks import CSVLogger
from keras.applications.densenet import DenseNet169
from keras.applications.densenet import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, log_loss, roc_auc_score
from mlxtend.plotting import plot_confusion_matrix
from pickle import load

In [ ]:
IMAGE_SIZE = [224, 224]
train_path = '../input/final-dataset-28/Mushroom_Classification_28/train'
val_path = '../input/final-dataset-28/Mushroom_Classification_28/val'
test_path = '../input/final-dataset-28/Mushroom_Classification_28/test'

In [ ]:
model = Sequential()

pretrained_model= tf.keras.applications.DenseNet169(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',classes=3,
                   weights='imagenet')

for layer in pretrained_model.layers:
        layer.trainable=False

model.add(pretrained_model)
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
  metrics=['accuracy']
)

In [ ]:
csv_logger = CSVLogger('/kaggle/working/densenet169_results.csv')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    rescale = 1./255)
   
test_datagen = ImageDataGenerator(
    rescale = 1./255)

train_set = train_datagen.flow_from_directory(train_path,
                                              target_size = (224, 224),
                                              batch_size = 32,
                                              class_mode = 'categorical',
                                              shuffle=True)

val_set = val_datagen.flow_from_directory(val_path,
                                          target_size = (224, 224),
                                          batch_size = 32,
                                          class_mode = 'categorical',
                                          shuffle=True)

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 1,
                                            class_mode = 'categorical',
                                            shuffle=False)

In [ ]:
STEPS_PER_EPOCH = train_set.n//train_set.batch_size
VALIDATION_STEPS = val_set.n//val_set.batch_size

In [ ]:
r = model.fit_generator(
  train_set,
  validation_data=val_set,
  epochs=100,
  steps_per_epoch=STEPS_PER_EPOCH,
  validation_steps=VALIDATION_STEPS,
  verbose=1,
  callbacks=[csv_logger])

In [ ]:
test_loss, test_acc = model.evaluate(test_set, verbose=1)

In [ ]:
num_of_test_samples = test_set.n
batch_size = test_set.batch_size

Y_pred = model.predict_generator(test_set, num_of_test_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
conf_matrix = confusion_matrix(test_set.classes, y_pred)
cm = np.array2string(conf_matrix)
fig, ax = plot_confusion_matrix(conf_mat=conf_matrix)
ax.set_title('Confusion Matrix\n');
plt.show()

In [ ]:
print('Classification Report')
target_names = ['Edible','Inedible','Poisonous']
class_report = classification_report(test_set.classes, y_pred, target_names=target_names)
print(class_report)

In [ ]:
print('Log Loss')
l_loss = log_loss(test_set.classes, Y_pred)
print(l_loss)

In [ ]:
print('ROC AUC Score')
r_a_score = roc_auc_score(test_set.classes, Y_pred, multi_class='ovr')
print(r_a_score)

In [ ]:
model.save('/kaggle/working/DenseNet169.h5')

In [ ]:
DenseNet169 = load_model('/kaggle/working/DenseNet169.h5')

In [ ]:
from keras.preprocessing import image
import os
from os import listdir

folder_dir = "../input/final-test-data/Test_Data/Inedible"
for images in os.listdir(folder_dir):
    if (images.endswith(".jpg")):
        im = plt.imread("../input/final-test-data/Test_Data/Inedible/"+images)
        plt.imshow(im)
        plt.show()
        output=[]
        img = image.load_img(os.path.join("../input/final-test-data/Test_Data/Inedible",images),target_size=(224,224))
        img = np.asarray(img)
        img = np.expand_dims(img, axis=0)
        output = model.predict(img)
    
        if output[0][0] > output[0][1] and output[0][0] > output[0][2]:
            print("Edible")
        elif output[0][1] > output[0][0] and output[0][1] > output[0][2]:
            print('Inedible')
        elif output[0][2] > output[0][0] and output[0][2] > output[0][1]:
            print('Poisonous')